# Baseline

A very simple approach

In [7]:
import os
import sys
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt

DATAPATH = "./data"

In [8]:
def mse(img1, img2):
    img1 = img1 / 255
    img2 = img2 / 255
    return np.sum(np.power(img1 - img2, 2))

In [9]:
def segment(img, checked, segmentation, x, y):
    if x < 0 or x >= img.shape[0] or y < 0 or y >= img.shape[1]:
        return 0
    if img[x, y] < 255 or checked[x, y]:
        return 0
    checked[x, y] = 1
    segmentation[x, y] = COUNTER
    return 1 + segment(img, checked, segmentation, x + 1, y) + \
            segment(img, checked, segmentation, x - 1, y) + \
            segment(img, checked, segmentation, x, y + 1) + \
            segment(img, checked, segmentation, x, y - 1)  

In [10]:
def get_colour(icon):
    white = np.where(np.all(255 - icon, axis=2))
    black = np.where(np.all(icon, axis=2))
    
    good1 = set()
    good2 = set()
    for i in range(len(black[0])):
        good1.add((black[0][i], black[1][i]))
    for i in range(len(white[0])):
        good2.add((white[0][i], white[1][i]))
    good = good1.intersection(good2)
    x, y = np.random.randint(120), np.random.randint(120)
    while (x, y) not in good:
        x, y = np.random.randint(120), np.random.randint(120)
    return icon[x, y]

Calculate the PSNR of the baseline

In [11]:
# Borrowed from https://dsp.stackexchange.com/questions/38065/peak-signal-to-noise-ratio-psnr-in-python-for-an-image
# Credit to user Himanshu Tyagi
def psnr(img1, img2):
    mse = np.mean( (img1 - img2) ** 2 )
    if mse == 0:
        return 100
    PIXEL_MAX = 1
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))


In [13]:
loss = 0
for t in range(1, 10001):
    edge = cv2.imread(f"{DATAPATH}/edges/{t}.png", 0)
    icon = cv2.imread(f"{DATAPATH}/icons/{t}.png")

    imgblur = cv2.GaussianBlur(edge, (3,3), 1)

    sys.setrecursionlimit(edge.shape[0] * edge.shape[1])
    seg = np.zeros(edge.shape)
    checked = np.zeros(edge.shape)
    segmentation = np.zeros(edge.shape)
    COUNTER = 1
    
    for x in range(edge.shape[0]):
        for y in range(edge.shape[1]):
            s = segment(imgblur, checked, segmentation, x, y)
            if s > 0:
                COUNTER += 1

    new_edge = cv2.cvtColor(edge, cv2.COLOR_GRAY2RGB)
    visited = set()
    black = np.where(segmentation == 0)
    white = np.where(segmentation == 1)
    for i in range(len(black[0])):
        visited.add((black[0][i], black[1][i]))
    for i in range(len(white[0])):
        visited.add((white[0][i], white[1][i]))

    losses = []
    for x in range(5):
        baseline = cv2.cvtColor(edge, cv2.COLOR_GRAY2RGB)
        for c in range(2, COUNTER):
            coords = np.where(segmentation == c)
            coords = np.stack(coords)
            colour = get_colour(icon)
            for i in range(len(coords[0])):
                x, y = tuple(coords[:, i])
                baseline[x, y] = colour
        baseline = cv2.GaussianBlur(baseline, (3, 3), 1)
        losses.append(psnr(baseline, icon))
    loss += max(losses)
    if t % 500 == 0:
        print(f"Current loss: {loss / t}")

KeyboardInterrupt: 